## Preamble

### Project Template

In [ ]:
%load_ext autoreload

In [ ]:
import os as _os

_os.chdir(_os.environ["PROJECT_ROOT"])
_os.path.realpath(_os.path.curdir)

### Imports

In [ ]:
import os
import subprocess
import sys
import time
from datetime import datetime
from glob import glob
from itertools import chain, product
from tempfile import mkstemp

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from scipy.spatial.distance import pdist, squareform
import seaborn as sns
import sfacts as sf
import statsmodels.api as sm
import statsmodels.formula.api as smf
import xarray as xr
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mtick
import mpltern

from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm

import lib.plot
from lib.dissimilarity import load_dmat_as_pickle
from lib.pandas_util import align_indexes, aligned_index, idxwhere, invert_mapping

In [ ]:
def linkage_order(linkage, labels):
    return labels[sp.cluster.hierarchy.to_tree(linkage).pre_order(lambda x: x.id)]

def plot_stacked_barplot(data, x_var, order, palette=None, ax=None, **kwargs):
    if ax is None:
        ax = plt.subplot()
    if palette is None:
        palette = lib.plot.construct_ordered_palette(order)

    # Bar styles
    bar_kwargs = dict(
        width=1.0,
        alpha=1.0,
        edgecolor="k",
        lw=1,
    )
    bar_kwargs.update(kwargs)

    # Plot each bar segment
    _last_top = 0
    for y_var in order:
        ax.bar(
            x=data[x_var],
            height=data[y_var],
            bottom=_last_top,
            label=y_var,
            color=palette[y_var],
            **bar_kwargs,
        )
        _last_top += data[y_var]
    ax.set_xticks(data[x_var].values)
    return ax


plot_stacked_barplot(
    pd.DataFrame(dict(t=[0, 1, 2], y1=[0.0, 0.5, 1.0], y2=[1.0, 0.5, 0.0])),
    x_var="t",
    order=["y1", "y2"],
)

In [ ]:
import lib.thisproject.data

### Set Style

In [ ]:
sns.set_context("talk")
plt.rcParams["figure.dpi"] = 100

## Metadata

In [ ]:
mgen = pd.read_table("meta/ucfmt/mgen.tsv", index_col="mgen_id")
sample = pd.read_table("meta/ucfmt/sample.tsv", index_col="sample_id")
subject = pd.read_table("meta/ucfmt/subject.tsv", index_col="subject_id")
assert mgen.sample_id.isin(sample.index).all()

mgen_meta = mgen.join(sample, on="sample_id").join(subject, on="subject_id")

### StrainPGC Post-filtering Strains

In [ ]:
spgc_meta = pd.read_table(f'data/group/ucfmt/species/sp-102506/r.proc.gtpro.sfacts-fit.gene99_v15-v22-agg75.spgc-fit.strain_meta_spgc_and_ref.tsv', index_col='genome_id')
spgc_strain_list = spgc_meta[lambda x: x.passes_filter & x.genome_type.isin(['SPGC'])].index.astype(int).values
len(spgc_strain_list)

## Strain tracking

In [ ]:
strain_fit = sf.data.World.load('data/group/ucfmt/species/sp-102506/r.proc.gtpro.sfacts-fit.world.nc').rename_coords(strain=str)

np.random.seed(0)
position_ss = strain_fit.random_sample(position=1000).position
sample_linkage = strain_fit.unifrac_linkage(optimal_ordering=True)
sample_linkage_alt = strain_fit.metagenotype.linkage(optimal_ordering=True)
strain_linkage = strain_fit.genotype.linkage(optimal_ordering=True)
position_linkage = strain_fit.metagenotype.sel(position=position_ss).linkage('position')

print(sf.evaluation.metagenotype_error2(strain_fit, discretized=True)[0])

### Visualize strain tracking

In [ ]:
focal_strains = ['6', '9', '33',
                 '38',  # Unfortunately low-quality gene content inferences. :-/
                ]  # Defined based on dominant, donor strains in followup samples.

In [ ]:
strain_order = [s for s in 
    linkage_order(
        strain_linkage,
        strain_fit.strain.values,
    )]
    # ) if s in spgc_strain_list]

if "-1" in strain_order:
    strain_order.remove("-1")  # Drop "other" strain.
strain_order.append("-1")  # Add to end of list

strain_palette = lib.plot.construct_ordered_palette(
    strain_order,  # Linkage order, I believe
    cm="turbo", vmin=0.05, vmax=0.95,
    extend={"-1": "silver"},
    desaturate_levels=[1.0],
)

# Desaturate non-focal strains
for strain in strain_palette:
    if strain == "-1":
        continue
    if strain not in focal_strains:
        strain_palette[strain] = sns.set_hls_values(strain_palette[strain], l=0.35, s=0.7)
    else:
        strain_palette[strain] = sns.set_hls_values(strain_palette[strain], l=0.75, s=1.0)

In [ ]:
strain_colors = pd.Series(strain_order, index=strain_order).map(strain_palette).to_frame()

sf.plot.plot_metagenotype(strain_fit.sel(position=position_ss), col_linkage=sample_linkage_alt)
sf.plot.plot_community(strain_fit.sel(position=position_ss), col_linkage=sample_linkage_alt, row_linkage=strain_linkage, row_colors=strain_colors)

In [ ]:
sf.plot.plot_genotype(strain_fit.sel(position=position_ss), row_linkage=strain_linkage, row_colors=strain_colors)

In [ ]:
strain_samples = pd.read_table('data/group/ucfmt/species/sp-102506/r.proc.gtpro.sfacts-fit.spgc_ss-all.strain_samples.tsv', names=['sample', 'strain'], dtype=str)
strain_samples[lambda x: x.strain.isin(focal_strains)].sort_values('strain').strain.value_counts()

In [ ]:
focal_strain_samples = idxwhere((strain_fit.community.data.sel(strain=focal_strains).max("strain") > 0.75).to_series())
w = strain_fit.sel(position=position_ss, sample=focal_strain_samples).drop_low_abundance_strains(0.001)
sf.plot.plot_community(w, col_linkage_func=lambda w: w.metagenotype.linkage())
sf.plot.plot_metagenotype(w, row_linkage_func=lambda w: position_linkage, col_linkage_func=lambda w: w.metagenotype.linkage())

In [ ]:
sample_type_specific_relabel = {
    'donor_enema': 'D',
    'donor_initial': 'D',
    'donor_capsule': 'D',
    'baseline': 'B',
    'post_antibiotic': 'pA',
    'pre_maintenance_1': 'M1',
    'pre_maintenance_2': 'M2',
    'pre_maintenance_3': 'M3',
    'pre_maintenance_4': 'M4',
    'pre_maintenance_5': 'M5',
    'pre_maintenance_6': 'M6',
    'followup_1': 'F1',
    'followup_2': 'F2',
    'followup_3': 'F3',
}

In [ ]:
for strain, c in strain_colors.squeeze().items():
    plt.scatter([], [], c=c, label=strain)
plt.legend(ncols=3)
lib.plot.hide_axes_and_spines()

In [ ]:
_meta = mgen_meta
subject_order = ["D0044", "D0097", "D0485"]

fig, axs = plt.subplots(1, len(subject_order), figsize=(5 * len(subject_order), 2), squeeze=False)
for subject_id, ax in zip(subject_order, axs.flatten()):
    sample_list = (
        _meta[
            lambda x: (x.subject_id == subject_id)
            & (x.index.isin(strain_fit.sample.values))
        ]
        # .sort_values('collection_days_post_fmt')  # This will be useful for subjects, but not donors
        .index
    )

    if len(sample_list) < 2:
        sample_order = sample_list
    else:
        sample_order = list(
            linkage_order(
                strain_fit.sel(sample=sample_list).metagenotype.linkage(
                    optimal_ordering=True
                ),
                sample_list,
            )
        )

    if len(sample_list) < 1:
        subject_comm = pd.DataFrame([], columns=[-1])
    else:
        subject_comm = (
            strain_fit.sel(sample=sample_order).keep_only_strain_list(strain_order)
            # .drop_low_abundance_strains(
            #     0.0, agg_strain_coord=-1
            # )  # TODO: Check that this adds to the alread-existing strain_-1
            .community.to_pandas()
        )

    d = (
        _meta.reindex(sample_order)
        # .dropna(subset=["collection_date_relative_een_end"])
        # .sort_values("collection_date_relative_een_end")
        .assign(
            t=lambda x: range(len(x)),
        )
    ).join(subject_comm)
    # d.loc[d.index[:num_offset_samples], 't'] -= 0.7  # Offset width

    plot_stacked_barplot(
        data=d,
        x_var="t",
        order=[s for s in strain_order if s in subject_comm.columns],
        palette=strain_palette,
        ax=ax,
        width=0.8,
        lw=0.5,
    )

    ax.set_title(subject_id)
    ax.set_xticklabels(
        d.sample_type_specific.map(sample_type_specific_relabel),
        fontsize=12,
    )
    ax.set_aspect(9, anchor="NW")
    ax.set_ylim(0, 1.0)
    lib.plot.rotate_xticklabels(rotation=90, ax=ax, ha="center")
    ax.set_yticks(np.linspace(0, 1.0, num=3))
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, symbol="%"))
    if not d.empty:
        ax.set_xlim(d.t.min() - 0.5, d.t.max() + 0.5)
    ax.spines[["right", "top"]].set_visible(False)

In [ ]:
donor_subject_order = ['D0044', 'D0097', 'D0485']
_meta = mgen_meta.loc[strain_fit.sample.values][lambda x:
    x.recipient
    # & ~x.sra_accession.isna()
]

for donor in donor_subject_order:
    subject_order = _meta[lambda x: x.donor_subject_id == donor].subject_id.value_counts().index    
    fig, axs = plt.subplots(1, len(subject_order), figsize=(3 * len(subject_order), 2), squeeze=False, sharey=True)
    for subject_id, ax in zip(subject_order, axs.flatten()):
        sample_order = (
            _meta[
                lambda x: (x.subject_id == subject_id)
                & (x.index.isin(strain_fit.sample.values))
            ]
            .sort_values('collection_days_post_fmt')
            .index
        )
    
        if len(sample_order) < 1:
            subject_comm = pd.DataFrame([], columns=[-1])
        else:
            subject_comm = (
                strain_fit.sel(sample=sample_order).keep_only_strain_list(strain_order)
                # .drop_low_abundance_strains(
                #     0.0, agg_strain_coord=-1
                # )  # TODO: Check that this adds to the alread-existing strain_-1
                .community.to_pandas()
            )
    
        d = (
            _meta.reindex(sample_order)
            # .dropna(subset=["collection_date_relative_een_end"])
            # .sort_values("collection_date_relative_een_end")
            .assign(
                t=lambda x: range(len(x)),
            )
        ).join(subject_comm)
        # d.loc[d.index[:num_offset_samples], 't'] -= 0.7  # Offset width

        plot_stacked_barplot(
            data=d,
            x_var="t",
            order=[s for s in strain_order if s in subject_comm.columns],
            palette=strain_palette,
            ax=ax,
            width=0.8,
            lw=0.5,
        )
    
        ax.set_title(subject_id)
        ax.set_xticklabels(
            d.sample_type_specific.map(sample_type_specific_relabel),
            fontsize=12,
        )
        ax.set_aspect(10, anchor="NW")
        ax.set_ylim(0, 1.0)
        lib.plot.rotate_xticklabels(rotation=90, ax=ax, ha="center")
        ax.set_yticks(np.linspace(0, 1.0, num=3))
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, symbol="%"))
        if not d.empty:
            ax.set_xlim(d.t.min() - 0.5, d.t.max() + 0.5)
        ax.spines[["right", "top"]].set_visible(False)

## Dominant strains from donors / recipients

In [ ]:
spgc_meta[lambda x: x.passes_filter & (x.genome_type == 'SPGC')]

In [ ]:
spgc_meta.loc[np.array(focal_strains).astype(str)]

## Compare to reference database? (maybe?)

In [ ]:
pd.read_table('ref/midasdb_uhgg_v15/metadata/genomes-all_metadata.tsv').columns

In [ ]:
ref_meta = pd.read_table(
    "ref/midasdb_uhgg_v15/metadata/2023-11-11-genomes-all_metadata.tsv",
    names=[
        "Genome",
        "Genome_type",
        "Length",
        "N_contigs",
        "N50",
        "GC_content",
        "Completeness",
        "Contamination",
        "rRNA_5S",
        "rRNA_16S",
        "rRNA_23S",
        "tRNAs",
        "Genome_accession",
        "Species_rep",
        "Lineage",
        "Sample_accession",
        "Study_accession",
        "Country",
        "Continent",
        "FTP_download",
        "_20",
        "_21",
    ],
).set_index('Genome_accession')
ref_meta

In [ ]:
geno_pdmat = lib.dissimilarity.load_dmat_as_pickle('data/group/ucfmt/species/sp-102506/r.proc.gtpro.sfacts-fit.spgc_ss-all.geno_uhgg-v15_pdist-mask10-pseudo10.pkl')
d = geno_pdmat.loc[spgc_meta.genome_type.isin(['Isolate', 'MAG']), all_strains]
pd.DataFrame(dict(min_diss=d.min(), ref=d.idxmin())).assign(Sample_accession=lambda x: ref_meta.loc[x.ref].Sample_accession.values)

## Inter-strain differences

## Differential Gene Content

## Gene Annotations

In [ ]:
gene_content_uhgg = pd.read_table('data/group/ucfmt/species/sp-102506/r.proc.gtpro.sfacts-fit.gene99_v15-v22-agg75.spgc-fit.uhgg-strain_gene.tsv', index_col='gene_id').astype(bool)
gene_content_uhgg = gene_content_uhgg.drop(idxwhere(gene_content_uhgg.sum(1) == 0))

In [ ]:
gene_content_eggnog = pd.read_table('data/group/ucfmt/species/sp-102506/r.proc.gtpro.sfacts-fit.gene99_v15-v22-agg75.spgc-fit.eggnog-strain_gene.tsv', index_col='gene_id').astype(bool)
gene_content_eggnog = gene_content_eggnog.drop(idxwhere(gene_content_eggnog.sum(1) == 0))

In [ ]:
gene_content_uhgg[focal_strains][lambda x: x.sum(1) > 0].value_counts().sort_index()

In [ ]:
gene_prevalence_ref_eggnog = pd.read_table('data/species/sp-102506/midasdb.gene75_v15.eggnog-strain_gene.prevalence.tsv', names=['gene_id', 'prevalence'], index_col='gene_id').prevalence
gene_prevalence_hmp2_eggnog = pd.read_table('data/group/hmp2/species/sp-102506/r.proc.gtpro.sfacts-fit.gene99_v15-v22-agg75.spgc-fit.eggnog-strain_gene.prevalence.tsv', names=['gene_id', 'prevalence'], index_col='gene_id').prevalence

In [ ]:
gene_prevalence_ref_uhgg = pd.read_table('data/species/sp-102506/midasdb.gene75_v15.uhgg-strain_gene.prevalence.tsv', names=['gene_id', 'prevalence'], index_col='gene_id').prevalence
gene_prevalence_hmp2_uhgg = pd.read_table('data/group/hmp2/species/sp-102506/r.proc.gtpro.sfacts-fit.gene99_v15-v22-agg75.spgc-fit.uhgg-strain_gene.prevalence.tsv', names=['gene_id', 'prevalence'], index_col='gene_id').prevalence

In [ ]:
d0 = gene_prevalence_hmp2_eggnog.to_frame().join(gene_content_eggnog).groupby(['6', '9']).prevalence

fig, axs = plt.subplots(2, 2, sharex=True, sharey=True)
bins = np.linspace(0, 1, num=11)
for (focal_strain_gene_type, (k, c)), ax in zip({'neither': ((False, False), 'silver'), 'D44': ((True, False), strain_palette['6']), 'D97': ((False, True), strain_palette['9']), 'both': ((True, True), 'black')}.items(), axs.T.flatten()):
    d1 = d0.get_group(k)
    ax.set_title(focal_strain_gene_type)
    ax.hist(d1, bins=bins, density=True, label=focal_strain_gene_type, color=c, alpha=0.5, histtype='stepfilled')
    ax.hist(d1, bins=bins, density=True, label=focal_strain_gene_type, color=c, alpha=0.85, histtype='step')
    # ax.annotate(len(d1), xy=(0.5, 0.5), xycoords='axes fraction')

fig.tight_layout()

In [ ]:
d0 = gene_prevalence_hmp2_uhgg.to_frame().join(gene_content_uhgg).groupby(['6', '9']).prevalence

fig, axs = plt.subplots(2, 2, sharex=True, sharey=True)
bins = np.linspace(0, 1, num=11)
for (focal_strain_gene_type, (k, c)), ax in zip({'neither': ((False, False), 'silver'), 'D44': ((True, False), strain_palette['6']), 'D97': ((False, True), strain_palette['9']), 'both': ((True, True), 'black')}.items(), axs.T.flatten()):
    d1 = d0.get_group(k)
    ax.set_title(focal_strain_gene_type)
    ax.hist(d1, bins=bins, density=True, label=focal_strain_gene_type, color=c, alpha=0.5, histtype='stepfilled')
    ax.hist(d1, bins=bins, density=True, label=focal_strain_gene_type, color=c, alpha=0.85, histtype='step')
    if not focal_strain_gene_type == 'neither':
        ax.annotate(len(d1), xy=(0.5, 0.5), xycoords='axes fraction')

fig.tight_layout()

In [ ]:
low_prevalence_genes_in_all_focal_strains = idxwhere(gene_prevalence_hmp2_uhgg[idxwhere(gene_content_uhgg[focal_strains].all(1))] < 0.2)
low_prevalence_genes_in_all_focal_strains

In [ ]:
from matplotlib_venn import venn2

venn2([set(idxwhere(gene_content_uhgg['6'])),set(idxwhere(gene_content_uhgg['9']))], set_labels=['D44', 'D97'], set_colors=[strain_palette['6'], strain_palette['9']])

In [ ]:
gene_clust = pd.read_table('data/group/hmp2/species/sp-102506/r.proc.gtpro.sfacts-fit.gene99_v15-v22-agg75.spgc-fit.uhgg-strain_gene.gene_clust-t10.tsv', names=['gene_id', 'clust'], index_col='gene_id').clust
gene_clust.value_counts()

In [ ]:
gene_content_clust = (
    gene_content_uhgg.groupby(gene_clust)
    .mean()
    .drop([-1, -2, -3, -4], errors="ignore")
    .drop(idxwhere(gene_clust.value_counts() <= 1), errors="ignore")
)

In [ ]:
# Presence of an HMP2/SPGC defined gene cluster requires 80% of the genes in it to be found in the strain

venn2([set(idxwhere(gene_content_clust['6'] > 0.8)),set(idxwhere(gene_content_clust['9'] > 0.8))], set_labels=['D44', 'D97'], set_colors=[strain_palette['6'], strain_palette['9']])

In [ ]:
clust_d44 = set(idxwhere(gene_content_clust['6'] > 0.8))
clust_d97 = set(idxwhere(gene_content_clust['9'] > 0.8))

In [ ]:
pd.DataFrame(
    dict(
        clust_size=gene_clust.value_counts(),
        d44_frac=gene_content_clust["6"],
        d97_frac=gene_content_clust["9"],
    )
)[lambda x: (x.d44_frac > 0.8) & (x.d97_frac > 0.8)].sort_values(
    "clust_size", ascending=False
).head(10)

In [ ]:
pd.DataFrame(
    dict(
        clust_size=gene_clust.value_counts(),
        d44_frac=gene_content_clust["6"],
        d97_frac=gene_content_clust["9"],
    )
)[lambda x: (x.d44_frac > 0.8) & (x.d97_frac <= 0.8)].sort_values(
    "clust_size", ascending=False
).head(10)

In [ ]:
pd.DataFrame(
    dict(
        clust_size=gene_clust.value_counts(),
        d44_frac=gene_content_clust["6"],
        d97_frac=gene_content_clust["9"],
    )
)[lambda x: (x.d44_frac <= 0.8) & (x.d97_frac > 0.8)].sort_values(
    "clust_size", ascending=False
).head(10)

In [ ]:
gene_annotation = pd.read_table('data/species/sp-102506/midasdb_v15.gene75_meta.tsv', index_col='gene_id')

In [ ]:
gene_x_cog = pd.read_table('data/species/sp-102506/midasdb_v15.emapper.gene75_x_cog.tsv').drop_duplicates()
cog_x_cog_pathway = pd.read_table('ref/cog-20.meta.tsv', encoding='latin1', names=['cog', 'cog_categories', 'description', 'preferred_name', 'cog_pathway', '_5', 'color_hex'], index_col='cog').cog_pathway

In [ ]:
gene_annotation.loc[idxwhere(gene_clust == 534)].join(gene_content_uhgg[['6', '9']])

In [ ]:
gene_annotation.loc[idxwhere(gene_clust == 3003)].join(gene_content_uhgg[['6', '9']])

In [ ]:
gene_annotation.loc[idxwhere(gene_clust == 656)].join(gene_content_uhgg[['6', '9']])

In [ ]:
gene_annotation.loc[low_prevalence_genes_in_all_focal_strains]

In [ ]:
hmp2_strain_meta = pd.read_table('data/group/hmp2/species/sp-102506/r.proc.gtpro.sfacts-fit.gene99_v15-v22-agg75.spgc-fit.strain_meta_for_analysis.tsv', index_col='genome_id')
hmp2_strain_list = idxwhere((hmp2_strain_meta.genome_type == 'SPGC') & hmp2_strain_meta.passes_filter)
len(hmp2_strain_list)

In [ ]:
hmp2_geno = sf.Metagenotype.load('data/group/hmp2/species/sp-102506/r.proc.gtpro.sfacts-fit.spgc_ss-all.mgtp.nc').rename_coords(sample=str).sel(sample=hmp2_strain_list).to_estimated_genotype().discretized(max_ambiguity=0.1).rename_coords(strain=lambda s: f'hmp2_{s}')
ucfmt_geno = sf.Metagenotype.load('data/group/ucfmt/species/sp-102506/r.proc.gtpro.sfacts-fit.spgc_ss-all.mgtp.nc').rename_coords(sample=str).to_estimated_genotype().discretized(max_ambiguity=0.1)

combined_geno = sf.Genotype.concat(dict(hmp2=hmp2_geno, ucfmt=ucfmt_geno.sel(strain=focal_strains)), dim='strain', rename=False)


In [ ]:
from scripts.calculate_pairwise_genotype_masked_hamming_distance import native_masked_hamming_distance_pdist

combined_mgtp_geno_pdist = pd.DataFrame(squareform(native_masked_hamming_distance_pdist(combined_geno.values, pseudo=1)), index=combined_geno.strain, columns=combined_geno.strain)
combined_mgtp_geno_linkage = sp.cluster.hierarchy.linkage(squareform(combined_mgtp_geno_pdist), method='average', optimal_ordering=True)

In [ ]:
sf.plot.plot_genotype(combined_geno.sel(position=position_ss), row_linkage_func=lambda w: combined_mgtp_geno_linkage, col_linkage_func=lambda w: position_linkage, row_colors=strain_colors)

In [ ]:
gene_content_hmp2_uhgg = pd.read_table('data/group/hmp2/species/sp-102506/r.proc.gtpro.sfacts-fit.gene99_v15-v22-agg75.spgc-fit.uhgg-strain_gene.tsv', index_col='gene_id')[hmp2_strain_list].astype(bool).rename(columns=lambda s: f'hmp2_{s}')
gene_content_ucfmt_uhgg = gene_content_uhgg[focal_strains]  # pd.read_table('data/group/ucfmt/species/sp-102506/r.proc.gtpro.sfacts-fit.gene99_v15-v22-agg75.spgc-fit.uhgg-strain_gene.tsv', index_col='gene_id').astype(bool)
gene_content_combined = pd.concat([gene_content_hmp2_uhgg, gene_content_ucfmt_uhgg], axis='columns').fillna(False)

In [ ]:
d = gene_content_combined[
    lambda x: (gene_prevalence_hmp2_uhgg > 0.15) & (gene_prevalence_hmp2_uhgg < 0.9)
]

cg = sns.clustermap(
    d,
    col_linkage=combined_mgtp_geno_linkage,
    metric="cosine",
    xticklabels=False,
    yticklabels=False,
    figsize=(10, 4),
    tree_kws=dict(lw=1),
    dendrogram_ratio=0.3,
)
cg.ax_row_dendrogram.set_visible(False)
cg.ax_cbar.set_visible(False)

d.shape

In [ ]:
d = gene_content_combined[lambda x: (gene_prevalence_hmp2_uhgg > 0.15) & (gene_prevalence_hmp2_uhgg < 0.9)]

cg = sns.clustermap(d, col_linkage=combined_mgtp_geno_linkage, col_colors=strain_colors, metric='cosine', xticklabels=False, yticklabels=False, figsize=(10, 4), tree_kws=dict(lw=1),     dendrogram_ratio=0.3)
cg.ax_row_dendrogram.set_visible(False)
cg.ax_cbar.set_visible(False)

In [ ]:
combined_gene_content_pdist = lib.dissimilarity.dmatrix(gene_content_combined[
    lambda x: (gene_prevalence_hmp2_uhgg > 0.15) & (gene_prevalence_hmp2_uhgg < 0.9)
].T, metric='jaccard')

_strain_list = combined_gene_content_pdist.index
_x = squareform(combined_mgtp_geno_pdist.loc[_strain_list, _strain_list])
_y = squareform(combined_gene_content_pdist.loc[_strain_list, _strain_list])

sp.stats.pearsonr(_x, _y)

In [ ]:
sns.clustermap(combined_mgtp_geno_pdist, row_linkage=combined_mgtp_geno_linkage, col_linkage=combined_mgtp_geno_linkage, col_colors=strain_colors, row_colors=strain_colors, )

In [ ]:
gene_x_cog_category_matrix = pd.read_table('data/species/sp-102506/midasdb_v15.emapper.gene75_x_cog_category.tsv').assign(flag=True).set_index(['centroid_75', 'cog_category']).flag.unstack('cog_category', fill_value=False)

In [ ]:
d0 = gene_content_ucfmt_uhgg[focal_strains][lambda x: x.sum(1) > 0]
d1 = (
    d0.assign(
        prevalence=gene_prevalence_hmp2_uhgg,
        cat_n=gene_x_cog_category_matrix['N'],
        cat_x=gene_x_cog_category_matrix['X'],
        cat_u=gene_x_cog_category_matrix['U'],
        # cat_code=lambda x: x.cat_n * 1 + x.cat_x * 2
    )
    .fillna({'cat_n': False, 'cat_x': False, 'cat_u': False})
    .reset_index()
    # .set_index(focal_strains, drop=False)
    # [lambda x: x[focal_strains].apply(lambda y: (y.values == np.array((True, True, False, False))).all(), axis=1)]
    .groupby(focal_strains)
)

set_to_label = {(True, True, True, True): 'all_4_strains', (True, True, False, False): 'in_high_only', (False, False, True, True): 'not_in_high', (False, False, True, False): 'not_in_high', (False, False, False, True): 'not_in_high'}
d2 = pd.concat(d1.get_group(k).assign(label=label) for k, label in set_to_label.items()).assign(pos=lambda x: x.label.map({'all_4_strains': 0, 'in_high_only': 1, 'not_in_high': 2}))

cat_n_fraction = d2[['pos', 'cat_n']].value_counts() / d2[['pos']].value_counts()
cat_x_fraction = d2[['pos', 'cat_x']].value_counts() / d2[['pos']].value_counts()
cat_u_fraction = d2[['pos', 'cat_u']].value_counts() / d2[['pos']].value_counts()

fig, axs = plt.subplots(4, sharex=True, figsize=(6, 10), gridspec_kw=dict(height_ratios=(0.15, 0.15, 0.4, 0.25)))

sns.barplot(x='pos', y='count', data=d2['pos'].value_counts().reset_index(), ax=axs[3], color='grey', linewidth=2, edgecolor='black')
sns.boxenplot(x='pos', y='prevalence', data=d2, ax=axs[2], showfliers=False, color='grey', linewidth=2, linecolor='black')
sns.barplot(x='pos', y='count', data=cat_n_fraction.xs(True, level='cat_n').reset_index(), ax=axs[1], color='tab:blue', edgecolor='tab:blue', alpha=0.8, lw=2)
sns.barplot(x='pos', y='count', data=cat_x_fraction.xs(True, level='cat_x').reset_index(), ax=axs[0], color='tab:orange', edgecolor='tab:orange', alpha=0.8, lw=2)

for i in [0, 1]:
    axs[i].set_ylim(0, 0.15)

In [ ]:
d2.groupby('label').prevalence.quantile([0.25, 0.5, 0.75])

## HE-associated clusters

In [ ]:
d0 = gene_content_ucfmt_uhgg[focal_strains][lambda x: x.sum(1) > 0]
d1 = (
    d0.assign(
        prevalence=gene_prevalence_hmp2_uhgg,
        clust=gene_clust,
    )
    .reset_index()
    # .set_index(focal_strains, drop=False)
    # [lambda x: x[focal_strains].apply(lambda y: (y.values == np.array((True, True, False, False))).all(), axis=1)]
    .groupby(focal_strains)
)

set_to_label = {(True, True, True, True): 'all_4_strains', (True, True, False, False): 'in_high_only', (False, False, True, True): 'not_in_high', (False, False, True, False): 'not_in_high', (False, False, False, True): 'not_in_high'}
d2 = pd.concat(d1.get_group(k).assign(label=label) for k, label in set_to_label.items()).assign(pos=lambda x: x.label.map({'all_4_strains': 0, 'in_high_only': 1, 'not_in_high': 2}))

d2.groupby(['clust', 'label']).prevalence.agg(['median', 'count']).unstack('label', fill_value=0).reorder_levels([1, 0], axis='columns').sort_index(axis='columns').sort_values(('in_high_only', 'count'), ascending=False).assign(clust_size=gene_clust.value_counts()).head(10)

In [ ]:
clust = 2594

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

In [ ]:
clust = 805

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

In [ ]:
clust = 1751

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

In [ ]:
clust = 806

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

In [ ]:
clust = 1250

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

In [ ]:
clust = 2808

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

In [ ]:
clust = 2891

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

In [ ]:
clust = 2820

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

## D44 vs. D97 HE-associated clusters

In [ ]:
gene_content_ucfmt_uhgg[['6', '9']].value_counts()

### D44

In [ ]:
d0 = gene_content_ucfmt_uhgg[['6', '9']][lambda x: x.sum(1) > 0]
d1 = (
    d0.assign(
        prevalence=gene_prevalence_hmp2_uhgg,
        clust=gene_clust,
    )
    .reset_index()
    # .set_index(focal_strains, drop=False)
    # [lambda x: x[focal_strains].apply(lambda y: (y.values == np.array((True, True, False, False))).all(), axis=1)]
    .groupby(['6', '9'])
)

set_to_label = {(True, False): 'd44_only', (False, True): 'd97_only'}
d2 = pd.concat(d1.get_group(k).assign(label=label) for k, label in set_to_label.items())

d2.groupby(['clust', 'label']).prevalence.agg(['median', 'count']).unstack('label', fill_value=0).reorder_levels([1, 0], axis='columns').sort_index(axis='columns').sort_values(('d44_only', 'count'), ascending=False).assign(clust_size=gene_clust.value_counts()).head(15)

In [ ]:
clust = 1252

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

In [ ]:
clust = 698

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

### D97

In [ ]:
d0 = gene_content_ucfmt_uhgg[['6', '9']][lambda x: x.sum(1) > 0]
d1 = (
    d0.assign(
        prevalence=gene_prevalence_hmp2_uhgg,
        clust=gene_clust,
    )
    .reset_index()
    # .set_index(focal_strains, drop=False)
    # [lambda x: x[focal_strains].apply(lambda y: (y.values == np.array((True, True, False, False))).all(), axis=1)]
    .groupby(['6', '9'])
)

set_to_label = {(True, False): 'd44_only', (False, True): 'd97_only'}
d2 = pd.concat(d1.get_group(k).assign(label=label) for k, label in set_to_label.items())

d2.groupby(['clust', 'label']).prevalence.agg(['median', 'count']).unstack('label', fill_value=0).reorder_levels([1, 0], axis='columns').sort_index(axis='columns').sort_values(('d97_only', 'count'), ascending=False).assign(clust_size=gene_clust.value_counts()).head(10)

In [ ]:
clust = 1102

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

In [ ]:
clust = 3003

print(gene_x_cog_category_matrix.reindex(idxwhere(gene_clust == clust)).sum().sort_values(ascending=False).head(5))
gene_annotation.loc[idxwhere(gene_clust == clust)].join(gene_content_uhgg[focal_strains]).sort_values('Preferred_name')

## GO-term enrichment

In [ ]:
gene_x_go = pd.read_table('data/species/sp-102506/midasdb_v15.emapper.gene75_x_go.tsv')
gene_x_go

In [ ]:
d0 = gene_content_ucfmt_uhgg[focal_strains][lambda x: x.sum(1) > 0]
d1 = (
    d0.assign(
        prevalence=gene_prevalence_hmp2_uhgg,
    ).groupby(focal_strains)
    # .reset_index()
    # .set_index(focal_strains, drop=False)
    # [lambda x: x[focal_strains].apply(lambda y: (y.values == np.array((True, True, False, False))).all(), axis=1)]
)

high_strain_gene = d1.get_group((True, True, False, False)).index
not_high_strain_gene = pd.concat([d1.get_group((False, False, True, True)), d1.get_group((False, False, True, False)), d1.get_group((False, False, False, True))]).index
print(len(high_strain_gene), len(not_high_strain_gene))

d2 = (
    gene_x_go.set_index(["centroid_75", "go"])
    .assign(flag=True)
    .flag.unstack("go", fill_value=False)
    .assign(
        which_strain_set=lambda x: np.where(
            x.index.isin(high_strain_gene),
            "high",
            np.where(x.index.isin(not_high_strain_gene), "not_high", np.nan),
        )
    )[lambda x: x.which_strain_set.isin(["high", "not_high"])]
)

go_term_list = idxwhere(d2.drop(['which_strain_set'], axis='columns').sum() > 3)
print(len(go_term_list))

results = []
for go_term in go_term_list:
    d3 = d2[[go_term, 'which_strain_set']].value_counts().unstack().reindex(index=[False, True], columns=['high', 'not_high']).fillna(0)
    contingency_pc = d3 + 1
    log2_odds_ratio_pc = np.log2((contingency_pc.loc[True, 'high'] / contingency_pc.loc[True, 'not_high']) / (contingency_pc.loc[False, 'high'] / contingency_pc.loc[False, 'not_high']))
    results.append((go_term, contingency_pc.loc[True, 'high'], log2_odds_ratio_pc, sp.stats.fisher_exact(d3)[1],))

results = pd.DataFrame(results, columns=['go_term', 'tally', 'log2_odds_ratio_pc', 'pvalue'])

In [ ]:
results.assign(fdr=lambda x: fdrcorrection(x.pvalue)[1]).sort_values('pvalue').head(20)

In [ ]:
d0 = (
    gene_x_cog_category_matrix
    .assign(
        which_strain_set=lambda x: np.where(
            x.index.isin(high_strain_gene),
            "high",
            np.where(x.index.isin(not_high_strain_gene), "not_high", np.nan),
        )
    )[lambda x: x.which_strain_set.isin(["high", "not_high"])]
)

results = []
for cog_category in gene_x_cog_category_matrix.columns:
    d1 = d0[[cog_category, 'which_strain_set']].value_counts().unstack().reindex(index=[False, True], columns=['high', 'not_high']).fillna(0)
    contingency_pc = d1 + 0
    log2_odds_ratio_pc = np.log2((contingency_pc.loc[True, 'high'] / contingency_pc.loc[True, 'not_high']) / (contingency_pc.loc[False, 'high'] / contingency_pc.loc[False, 'not_high']))
    results.append((cog_category, contingency_pc.loc[True, 'high'], log2_odds_ratio_pc, sp.stats.fisher_exact(d1)[1],))

results = pd.DataFrame(results, columns=['cog_category', 'tally', 'log2_odds_ratio_pc', 'pvalue']).assign(fdr=lambda x: fdrcorrection(x.pvalue)[1])

results.sort_values('pvalue').head(20)